In [1]:
import torch
from datatypes import *
from train_dm import *
import os
import json
import config
from collections import deque

In [2]:
path='example_outputs/run_262/'
outpath=path+'/eval.txt'
if os.path.exists(outpath):
    raise ValueError('Output file already exists')
with open(os.path.join(path, 'args.json'), 'r') as f:
    args = json.load(f)
sorted(os.listdir(path))

['H.pt',
 'args.json',
 'params_t-0.pt',
 'params_t-1.pt',
 'params_t-10.pt',
 'params_t-11.pt',
 'params_t-12.pt',
 'params_t-13.pt',
 'params_t-14.pt',
 'params_t-15.pt',
 'params_t-16.pt',
 'params_t-17.pt',
 'params_t-18.pt',
 'params_t-19.pt',
 'params_t-2.pt',
 'params_t-20.pt',
 'params_t-21.pt',
 'params_t-22.pt',
 'params_t-23.pt',
 'params_t-24.pt',
 'params_t-3.pt',
 'params_t-4.pt',
 'params_t-5.pt',
 'params_t-6.pt',
 'params_t-7.pt',
 'params_t-8.pt',
 'params_t-9.pt',
 'values.txt']

In [3]:
def getmodel(n,l):
    H = TFIM(n)
    ublock = UnitaryBlock(H, l=l, use_trotter=False, n=n).set_direction_forward()
    block = Block(unitaryblock=ublock)
    return block

def apply(model,param_blocks,n_superblocks,rho):
    yield cdot(psi_target, rho @ psi_target).real
    for _ in range(n_superblocks):
        for i,params in enumerate(param_blocks):
            rho=rho.detach()
            model.load_state_dict(params)
            rho=model.apply_to_density_matrix(rho)

        yield cdot(psi_target, rho @ psi_target).real

def retrieve(tensor):
    return tensor.detach().cpu().numpy()

In [4]:
block_param_files=[f for f in os.listdir(path) if 'params' in f]
block_param_files.sort(key=lambda s: int(s.split('-')[1].split('.')[0]))
block_param_files.reverse()
param_blocks=[torch.load(path+'/'+f) for f in block_param_files]

In [5]:
n=args['n']
l=args['l']
H=torch.load(path+'/H.pt')
psi_target = groundstate(H, n)
I = torch.eye(len(psi_target), dtype=tcomplex, device=config.device) / len(psi_target)
model=getmodel(n,l)

In [6]:

for d in range(len(block_param_files)+1):
    for s,out in enumerate(apply(model,param_blocks[len(param_blocks)-d:],24,I)):
        print('d',d,'superblock',s,'value',retrieve(out))

        with open(outpath, "a") as f:
            f.write(f"d {d} superblocks {s} value {retrieve(out)}\n")

    print()
        #print(out)I

d 0 superblock 0 value 0.0019531252
d 0 superblock 1 value 0.0019531252
d 0 superblock 2 value 0.0019531252
d 0 superblock 3 value 0.0019531252
d 0 superblock 4 value 0.0019531252
d 0 superblock 5 value 0.0019531252
d 0 superblock 6 value 0.0019531252
d 0 superblock 7 value 0.0019531252
d 0 superblock 8 value 0.0019531252
d 0 superblock 9 value 0.0019531252
d 0 superblock 10 value 0.0019531252
d 0 superblock 11 value 0.0019531252
d 0 superblock 12 value 0.0019531252
d 0 superblock 13 value 0.0019531252
d 0 superblock 14 value 0.0019531252
d 0 superblock 15 value 0.0019531252
d 0 superblock 16 value 0.0019531252
d 0 superblock 17 value 0.0019531252
d 0 superblock 18 value 0.0019531252
d 0 superblock 19 value 0.0019531252
d 0 superblock 20 value 0.0019531252
d 0 superblock 21 value 0.0019531252
d 0 superblock 22 value 0.0019531252
d 0 superblock 23 value 0.0019531252
d 0 superblock 24 value 0.0019531252

d 1 superblock 0 value 0.0019531252
d 1 superblock 1 value 0.0039058635
d 1 superblo